## Etapa de Transformação de Dados

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import re
import os
from pathlib import Path
import zipfile
import requests
#import io

home = Path.home()
correct_path = Path(home, "Documents")
    
# Downloading csv file from your GitHub account
url = "https://github.com/alemattos-cmd/rox_case/archive/refs/heads/main.zip" 
download = requests.get(url, allow_redirects=True)
with open(correct_path.joinpath("rox_case-main.zip"), "wb") as code:
    code.write(download.content)

# Extract data from zipfile
def un_zipFiles(correct_path):
    files=os.listdir(correct_path)
    for file in files:
        if file.endswith('.zip'):
            filePath=correct_path.joinpath(file)
            zip_file = zipfile.ZipFile(filePath)
            for names in zip_file.namelist():
                zip_file.extract(names,correct_path)
            zip_file.close()
        #file.close()
            
un_zipFiles(correct_path)

#update path
correct_path = Path(correct_path, "rox_case-main")

#### Preparação do arquivo 'Person'

In [2]:
df_person = pd.read_csv("Person.Person.csv", delimiter=";")
df_person = df_person.drop(columns=['NameStyle', 'Title', 'AdditionalContactInfo','rowguid', 'ModifiedDate'])

df_person.insert(8,'BirthDate', 'null', True)
df_person = df_person.astype("string")
df_person.insert(9,'TotalPurchase', 0, True)

df_person['BusinessEntityID'] = pd.to_numeric(df_person['BusinessEntityID'], errors='coerce')
df_person['EmailPromotion'] = pd.to_numeric(df_person['EmailPromotion'], errors='coerce')

#df_person.fillna(value='null')

t = df_person['Demographics'].values
d= t.copy()

## Extract TotalPurchase from a list of Demographics information
pattern = "<TotalPurchaseYTD>(.*?)</TotalPurchaseYTD>"
i=0
for s in t:
    totalpurchase = re.search(pattern, s).group(1)
    t[i] = totalpurchase
    i=i+1
    
df_person['TotalPurchase'] = t.astype("float64")

## Extract BirthDate from copy list of Demographics information
pattern2 = "<BirthDate>(.*?)Z</BirthDate>"
j=0
for g in d:
    if (re.search(pattern2, g)) != None:
        BirthDate=re.search(pattern2, g).group(1)
        d[j]=BirthDate
        j=j+1
    else:
        d[j] = 'null'
        j=j+1
        
df_person['BirthDate'] = d
df_person['BirthDate'] = pd.to_datetime(df_person['BirthDate'], errors='coerce')
df_person = df_person.drop(columns=['Demographics'])

# processing missing data
df_person.update(df_person['MiddleName'].fillna(''))
df_person.update(df_person['Suffix'].fillna(''))
#df_person.update(df_person['BirthDate'].fillna(''))

#print(df_person.isnull().sum())
#(df_person.info())
#display(df_person)
df_person.to_csv(correct_path.joinpath('Person.csv'),index=False)


#### Preparação do arquivo 'Customer'

In [3]:
df_customer = pd.read_csv("Sales.Customer.csv", delimiter=";" )
df_customer = df_customer.drop(columns=['rowguid', 'ModifiedDate'])

df_customer.fillna(value='null')
df_customer['AccountNumber'] = df_customer['AccountNumber'].astype("string")

#print(df_customer.isnull().sum())
#(df_customer.info())
#display(df_customer)

df_customer.to_csv(correct_path.joinpath('Customer.csv'),index=False)

#### Preparação do arquivo 'SalesOrderHeader'

In [4]:
df_header = pd.read_csv("Sales.SalesOrderHeader.csv", delimiter=";")
df_header = df_header.drop(columns=['Status', 'Comment','rowguid', 'ModifiedDate'])

df_customer.fillna(value='null')

c = ['CreditCardID', 'SalesPersonID', 'CurrencyRateID']
for colun in c:
    df_header[colun] = df_header[colun].apply(lambda x: int(x) if x == x else None)

df_header['OrderDate'] = pd.to_datetime(df_header['OrderDate'])
df_header['DueDate'] = pd.to_datetime(df_header['DueDate'])
df_header['ShipDate'] = pd.to_datetime(df_header['ShipDate'])
df_header['SalesOrderNumber'] = df_header['SalesOrderNumber'].astype("string")
df_header['PurchaseOrderNumber'] = df_header['PurchaseOrderNumber'].astype("string")
df_header['AccountNumber'] = df_header['AccountNumber'].astype("string")
df_header['CreditCardApprovalCode'] = df_header['CreditCardApprovalCode'].astype("string")


df_header["SubTotal"] = [(str(w).replace(" ", "")) for w in df_header["SubTotal"]]
df_header["SubTotal"] = [(str(w).replace("," , ".")) for w in df_header["SubTotal"]]
df_header['SubTotal'] = pd.to_numeric(df_header['SubTotal'], errors="coerce").map("{:.4f}".format).astype(float)

df_header["TaxAmt"] = [(str(w).replace(" ", "")) for w in df_header["TaxAmt"]]
df_header["TaxAmt"] = [(str(w).replace("," , ".")) for w in df_header["TaxAmt"]]
df_header['TaxAmt'] = pd.to_numeric(df_header['TaxAmt'], errors="coerce").map("{:.4f}".format).astype(float)

df_header["Freight"] = [(str(w).replace(" ", "")) for w in df_header["Freight"]]
df_header["Freight"] = [(str(w).replace("," , ".")) for w in df_header["Freight"]]
df_header['Freight'] = pd.to_numeric(df_header['Freight'], errors="coerce").map("{:.4f}".format).astype(float)

df_header["TotalDue"] = [(str(w).replace(" ", "")) for w in df_header["TotalDue"]]
df_header["TotalDue"] = [(str(w).replace("," , ".")) for w in df_header["TotalDue"]]
df_header['TotalDue'] = pd.to_numeric(df_header['TotalDue'], errors="coerce").map("{:.4f}".format).astype(float)

#print(df_header.isnull().sum())
#(df_header.info())
#display(df_header)
df_header.to_csv(correct_path.joinpath('SalesOrderHeader.csv'),index=False)

#### Preparação do arquivo 'SalesOrderDetail'

In [5]:
df_detail = pd.read_csv("Sales.SalesOrderDetail.csv", delimiter=";" )
df_detail = df_detail.drop(columns=['rowguid', 'ModifiedDate'])

df_detail['CarrierTrackingNumber'] = df_detail['CarrierTrackingNumber'].astype("string")
df_detail["UnitPrice"] = [(str(w).replace(" ", "")) for w in df_detail["UnitPrice"]]
df_detail["UnitPrice"] = [(str(w).replace("," , ".")) for w in df_detail["UnitPrice"]]
df_detail['UnitPrice'] = pd.to_numeric(df_detail['UnitPrice'], errors="coerce").map("{:.4f}".format).astype(float)

df_detail["UnitPriceDiscount"] = [(str(w).replace(" ", "")) for w in df_detail["UnitPriceDiscount"]]
df_detail["UnitPriceDiscount"] = [(str(w).replace("," , ".")) for w in df_detail["UnitPriceDiscount"]]
df_detail['UnitPriceDiscount'] = pd.to_numeric(df_detail['UnitPriceDiscount'], errors="coerce").map("{:.2f}".format).astype(float)

#print(df_detail.isnull().sum())
#(df_detail.info())
#display(df_detail)
df_detail.to_csv(correct_path.joinpath('SalesOrderDetail.csv'),index=False)

#### Preparação do arquivo 'SpecialOfferProduct'

In [6]:
df_special = pd.read_csv("Sales.SpecialOfferProduct.csv", delimiter=";" )
df_special = df_special.drop(columns=['rowguid', 'ModifiedDate'])

#(df_special.info())
#display(df_special)
df_special.to_csv(correct_path.joinpath('SpecialOfferProduct.csv'),index=False)

#### Preparação do arquivo 'Product'

In [7]:
df_product = pd.read_csv("Production.Product.csv", delimiter=";" )
df_product = df_product.drop(columns=['DiscontinuedDate','rowguid', 'ModifiedDate'])

c = ['Name', 'ProductNumber', 'Color', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'ProductLine', 'Class', 'Style']
for colun in c:
    df_product[colun] = df_product[colun].astype("string")
    

df_product["StandardCost"] = [(str(w).replace(" ", "")) for w in df_product["StandardCost"]]
df_product["StandardCost"] = [(str(w).replace("," , ".")) for w in df_product["StandardCost"]]
df_product['StandardCost'] = pd.to_numeric(df_product['StandardCost'], errors="coerce").map("{:.4f}".format).astype(float)

df_product["ListPrice"] = [(str(w).replace(" ", "")) for w in df_product["ListPrice"]]
df_product["ListPrice"] = [(str(w).replace("," , ".")) for w in df_product["ListPrice"]]
df_product['ListPrice'] = pd.to_numeric(df_product['ListPrice'], errors="coerce").map("{:.4f}".format).astype(float)

df_product['SellStartDate'] = pd.to_datetime(df_product['SellStartDate'], errors='coerce')
df_product['SellEndDate'] = pd.to_datetime(df_product['SellEndDate'], errors='coerce')

#print(df_product.isnull().sum())
#(df_product.info())
#display(df_product)
df_product.to_csv(correct_path.joinpath('Product.csv'),index=False)

print('Done')

Done
